In [ ]:
來源:SixSite_random.ipynb

In [1]:
import random
import requests
import pandas as pd

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By

import lxml
import time

from helpers import *

import os
import sqlite3

# base_domain_name_dict = {
#     0:'https://justdata.moneydj.com/', # MoneyDJ
#     1:'http://jdata.yuanta.com.tw/', # 元大
#     2:'https://fubon-ebrokerdj.fbs.com.tw/', # 富邦
#     3:'http://moneydj.emega.com.tw/', # 兆豐
#     4:'http://newjust.masterlink.com.tw/', # 元富
#     5:'https://djinfo.cathaysec.com.tw/' # v國泰金
#     }

base_domain_name_dict = {
    0:'https://justdata.moneydj.com/', # MoneyDJ
    1:'http://jdata.yuanta.com.tw/', # 元大
#    2:'https://fubon-ebrokerdj.fbs.com.tw/', # 富邦
    2:'http://moneydj.emega.com.tw/', # 兆豐
    3:'http://newjust.masterlink.com.tw/', # 元富
    4:'https://djinfo.cathaysec.com.tw/' # v國泰金
    }


# 一早就搞定urls的隨機問題
list_domain_name_remain = list(base_domain_name_dict.items())
random.shuffle(list_domain_name_remain)
#print(list_domain_name_remain)

print('請輸入股票代號 : ')
stock_id = input()

# random.choice從字典挑出來的，竟然是只挑value．而且是以位置(從0開始算)

# pop()會回傳 第某個位置的元素
value = list_domain_name_remain.pop(0)[1] # list_domain_name_remain中是含 key、value 的 六個item。

url = f'{value}z/zc/zco/zco_{stock_id}.djhtm'


print(url)

tuple_for_href = () # 這個應放在 Main Part 中，而非放在 方法中
print(list_domain_name_remain)
#########################################
# 二、取得 某一段期間 主力進出明細 所有券商的href
# (被二_1 呼叫)

def get_AllAgent_href(tuple_for_href): # 試著不傳入driver。結果是:正確的！！
                                                             # 此 driver 包含某一段期間的30家買超、賣超券商名稱
                                                             # url也不必傳入，
                                                         # 但tuple_for_href一定要傳入，很奇怪？
                                                             # url_different_period其實不用傳進來了？答案:不必傳進來！！
    list_hrefs = driver.find_elements_by_class_name('t4t1 a')
    
    for i in range(len(list_hrefs)):
        href = list_hrefs[i].get_attribute('href') # 此href已自動補上 domain_name！！
        
        # print('最開始tuple_for_href 的長度..', len(tuple_for_href))

        # 累加至變數 tuple_for_href 中
        tuple_for_href += (href,)
        
        # print('tuple_for_href 累加的長度...', len(tuple_for_href))
    
    # return driver, tuple_for_href 應該不需要 return driver
    return tuple_for_href
################################################
# 二_1、取得 主力進出明細 一、五、十、二十、四十、六十、一二〇、二四〇日，所有券商的href
# 在裡面要呼叫 二、
# def get_periods_AllAgent_href(tuple_for_href):
def get_periods_AllAgent_href(tuple_for_href): # 試著不必傳入全域變數tuple_for_href:會報錯！

    # for i in range(1,9): # 八個 periods
    for i in range(1,9): # 八個 periods

        url_different_period = url.replace(stock_id,stock_id+'_{}'.format(i))
        driver.get(url_different_period) # 此時driver已是第一個網站各個period的頁面的driver，呼叫二、時要用到。

        # 呼叫 二、get_AllAgent_href
        # driver, tuple_for_href = get_AllAgent_href(url_different_period, tuple_for_href, driver) 應該不必接收driver
        tuple_for_href = get_AllAgent_href(tuple_for_href) # 應該不必傳driver，因為方法內抓得到全域變數driver。

    href_set = set(tuple_for_href )
    href_list = list(href_set)

    print('len(href_set總共有多少個 href...', len(href_set))
    
    # 建立 href們的字典
    hrefs_dict = dict(zip(range(len(href_list)),href_list)) # zip()函式可以同時迭代多個序列，
                                                           # 序列可以是串列、tuple，返回的值是一個zip物件。
                                                           # 再用 dict()將之做成字典！
    print('hrefs_dict=', hrefs_dict)
    
    # return tuple_for_href, href_set # return出去，改變全域變數tuple_for_href
    return tuple_for_href, href_list, hrefs_dict # 若tuple_for_href沒有return出去，有改變全域變數 tuple_for_href？答案是:沒有改到。
####################################################
# 四、 取得 日期 收盤價 漲跌幅 的 df   (在 # 六、的廻圈外就先叫了！再於六的迴圈內 做merge！)
def get_date_close_UpDown(stock_id): # 從 yf 抓 個股歷史行情，簡捷！但當天收盤需至晚上12點才有
    import yfinance as yf
    import datetime

    df_source = yf.download('{}.TW'.format(stock_id), start='2018-1-1', end=datetime.date.today())

    if df_source.empty == True:

        df_source = yf.download('{}.TWO'.format(stock_id), start='2018-1-1', end=datetime.date.today())
        print('此股票為上櫃或興櫃 : ', df_source)    
    
    # 整個 df 頭尾列反轉

    df = df_source.iloc[::-1]
    # print('df :',df)

    # 令 index 變成 indexrange！！！
    df.reset_index(inplace=True) 

    # 增加 漲跌% 欄位 好像部分成功了！

    a = list(['{:.2%}'.format((float(df.loc[df.index==i, 'Close'])-float(df.loc[df.index==(i+1), 'Close']))/float(df.loc[df.index==(i+1),'Close'])) for i in range(len(df)-1)])
    a=a+['nan']
    df['漲跌%'] = a
    # print('漲跌',df)
    # 僅有 日期 收盤 漲跌%

    df.reset_index(drop=True, inplace=True)
    df.set_index(['Date'], inplace=True)
    df=df[['Close', '漲跌%']]

    # 取小數二位，變更欄位名稱，設index
    df=df.round(2)
    df.reset_index(inplace=True)
    df=df.rename(columns={'Date':'日期', 'Close':'收盤'})
    df.set_index('日期', inplace=True,drop=True)
    
    df_date_close_UpDown = df
    
    df_result = df_date_close_UpDown

    #df_result = df_date_close_UpDown # 因為改為先叫 # 四、故，直接把df_date_close_UpDown設給 result_df以便到 # 六、的後續處理！！
    
    return df_result
#################################################
# 五、專攻:只讀取 「第一個網站」的hrefs 做成 df_for_merge，並呼叫五_1、最後回傳未完成(或空的)的 新的hrefs_dict
# 此cell從六、中獨立出來實驗: 驗收合格，採用！
# 有未成功者，再呼叫 五_1直至全部成功 或 剩下五家網站全跑完

def build_df_for_merge(hrefs_dict, list_domain_name_remain, df_result): # list_domain_name_remain中是含 key、value 的 五個item
                                                                        # df_result是從 四、完成的前三欄。供 六_1合併使用
    keys_for_pop = []
    # 將20日改成 60日

    for key, value in hrefs_dict.items():# 此value是有含domain_name的 href(第一家網站/某分點/預設的二十日)
                                         # 範例:http://moneydj.emega.com.tw/z/zc/zco/zco0/zco0.djhtm?a=1342&b=0031003100360043&BHID=1160
        
        #### 重要，在這裡取得 stock_name_id、agent_name
        print('五、中，做driver.get(value)時的value是…', value)
        driver.get(value) # 進入到某券商分點，即將進行點擊 近60日 ！)
        if 'justdata.moneydj' in value or 'jdata.yuanta' in value or 'moneydj.emega' in value :
            driver.find_element_by_xpath('//*[@id="SysJustIFRAMEDIV"]/table/tbody/tr[2]/td[2]/center/form/table/tbody/tr/td/table/tbody/tr[5]/td/select[1]/option[4]').click()
#         elif 'fubon-ebrokerdj' in value:
#             try:
#                 driver.find_element_by_xpath('//*[@id="SysJustIFRAMEDIV"]/table/tbody/tr[2]/td[2]/center/form/table/tbody/tr/td/table/tbody/tr[5]/td/select[1]/option[4]').click()
#             except:
#                 driver.find_element_by_xpath('//*[@id="SysJustIFRAMEDIV"]/table/tbody/tr[2]/td[2]/table/tbody/tr/td/form/table/tbody/tr/td/table/tbody/tr[5]/td/select[1]/option[4]').click()

        elif 'newjust.masterlink' in value:
            driver.find_element_by_xpath('//*[@id="SysJustIFRAMEDIV"]/table/tbody/tr[2]/td[2]/center/table/tbody/tr/td/form/table/tbody/tr/td/table/tbody/tr[5]/td/select[1]/option[4]').click()
        else : # 'djinfo.cathaysec' in value:
            driver.find_element_by_xpath('//*[@id="SysJustIFRAMEDIV"]/table/tbody/tr[2]/td[2]/span/form/table/tbody/tr/td/table/tbody/tr[5]/td/select[1]/option[4]').click()

            #         else : # fubon-ebrokerdj
            #  driver.find_element_by_xpath('//*[@id="SysJustIFRAMEDIV"]/table/tbody/tr[2]/td[2]/table/tbody/tr/td/form/table/tbody/tr/td/table/tbody/tr[5]/td/select[1]/option[4]').click()

        html_source = driver.page_source
        try :
            dfs_for_merge = pd.read_html(html_source, header=[0])
            #print('11111:此行無誤dfs_for_merge = pd.read_html(html_source, header=[0]) ', dfs_for_merge)
            # 這裡似乎還要用程式處理？？
            # (算了，[3]不行，換其它網站找反而省事？不對，[3]找到的table不一定是所需的，還是逐一檢查好了)不同的網站，用小程式新開cell測試一次即可知道所需table在第？個 :value_counts_in_dfs(dfs,'110/08')

            if 'newjust.masterlink' in value :
                df_for_merge = dfs_for_merge[4] # fubon-ebrokerdj好像不是3這張table
            elif 'justdata.moneydj' in value :
                #print('222:有進入到此行 elif "justdata.moneydj" in value :')

                #print('2222:dfs_for_merge[-1].columns的長相:', dfs_for_merge[-1].columns)
                if '日期' in dfs_for_merge[-1].columns:
                    #print('22222:此行無誤 if "日期" in dfs_for_merge[-1].columns: ')
                    df_for_merge = dfs_for_merge[-1] # 也有可能是[3]這張table嗎？做個try？
                elif '日期' in dfs_for_merge[3].columns:
                    #print('33333:此行無誤 elif "日期" in dfs_for_merge[3].columns:')
                    df_for_merge = dfs_for_merge[3]
                else :
                    #print('444444: justdata.moneydj第{}筆查無資料了，用continue繼續下一個key'.format(key))
                    continue                
            else :
                df_for_merge = dfs_for_merge[3] # fubon-ebrokerdj好像不是3這張table
            
            #print('55555: 近六十日df_for_merge正確嗎？', df_for_merge) # 近六十日的 df
            
        except :
            #print('66666: 某分點的key:{}，在五、的時候，其 df 為空的……'.format(key))

            continue # 繼續下一分點

        #(暫停5) 用pop()去掉 try 中已成功的，最後剩下的再 retry
        # 逐次pop()改為收集在 keys_for_pop
        else :
            print('某分點的key:{}，df 已成功！！'.format(key))
            keys_for_pop.append(key)

            
#### 重要，在這裡取得 stock_name_id 及 agent_name
        title_agent = driver.find_element_by_class_name('t10').text
        # 答案應該是 元大-民生 對 八貫(1342)個股 單一券商歷史明細
        print('title_agent是', title_agent) # 例:元大-民生 對 八貫(1342)個股 單一券商歷史明細
        
        # 此處產生的stock_name_id是local variable 要return出去
        stock_name_id = title_agent.split(' ',3)[2].replace('(', '_').replace(')個股','')
        # 結果是 八貫_1342
        
        agent_name = title_agent.split(' ')[0]
        # 結果是 元大-民生
        
       ### 呼叫程式執行 合併至df_result、存檔(在這裡 第一輪網站的 某分點 已做完)

        #=======================================================================================================
        # 呼叫 # 六_1、各分點的df_for_merge存成csv檔，合併至df_result
        print('六_1、呼叫 sourtout_And_SaveToCvs()')

        # 上面已取得 stock_name_id
        df_SourtOut_SaveToCsv  = sourtout_And_SaveToCvs(df_for_merge, key, stock_name_id, agent_name)

        #========================================================================================================
        # 呼叫 # 六_2、前三欄 與 各券商分點資料 合併
        # print('六_2、呼叫 merge_dfs() 前三欄 與 各券商分點資料 合併 : ')
        df_result = merge_dfs(df_result, df_SourtOut_SaveToCsv, agent_name)

        # 隨機的休息
        # random = randint(4,22)
        # time.sleep(random)
        # time.sleep(randint(0,2)) 似乎抓太快在第33筆被擋住了，改回1~25秒

        print('迴圈 {}合併完成！！！！！休息1秒'.format(key))
        time.sleep(1)
    
    # for 迴圈結束=================================================    
           
    # 此時迴圈結束了，才來一次做pop()
    for key in keys_for_pop:
        hrefs_dict.pop(key)
 
    # 呼叫五_1 以迴圈方式直至全部成功 或 剩下五家網站全跑完
    
    if hrefs_dict : # 若第一個網站結束後hrefs_dict裡面有不成功的資料(即 True)，就呼叫 五_1
        
        # (有錯)for key in list(list_domain_name_remain.keys()): # list_domain_name_remain中是含 key、value 的 六個item
        # 在此做五個item的迴圈，所以 五_1中不必再將 五家list_domain_name_remain做 pop()
        for item in list_domain_name_remain: # list_domain_name_remain中是含 key、value 的 五個item 的 list。
                                            # 例:[(0, 'https://justdata.moneydj.com/'), (2, 'https://fubon-ebrokerdj.fbs.com.tw/')…
            value_of_domain_name_remain = item[1]
            
            # list_domain_name_remain, hrefs_dict = retry_WebSites(value_of_domain_name_remain, hrefs_dict) # 等號左、右兩邊的hrefs_dict，不斷反芻
            hrefs_dict, df_result = retry_WebSites(value_of_domain_name_remain, hrefs_dict, df_result, stock_name_id) # 等號左、右兩邊的hrefs_dict，不斷反芻

            # if not list_domain_name_remain or not hrefs_dict : # list_domain_name_remain為空的 或 hrefs_dict為空的 則結束 五_1、
            if not hrefs_dict : # hrefs_dict為空的(即 not False=True) 則結束 五_1、
                print('hrefs_dict已空，結束 五_1 ！！')
                break
            else :
                print('已完成五_1的 {}網站…繼續下一個網站…'.format(item))
                continue
                
        
            print('for迴圈跑五家網站就結束 五_1、')
            break # for迴圈跑五家網站就終止 五_1、(此break多此一舉？)


            # 隨機的休息
            # random = randint(4,22)
            # time.sleep(random)
            # time.sleep(randint(0,2)) 似乎抓太快在第33筆被擋住了，改回1~25秒

            print('迴圈 {}合併完成！！！！！休息{1}秒'.format(key))
            time.sleep(1)


    else :
        print('{}筆hrefs一次抓取成功！！'.format(key)) # key是指最後一筆的 key，即總數。
        
    print('五、完成後的df_result，待總量的排序')
    return list_domain_name_remain, hrefs_dict, df_result, stock_name_id # 若全部抓取完成(一次完成或多次才完成(包含跑完五家仍有不成功的))其實沒啥好回傳，除了stock_name_id？及…？？
                                        # 待六、的程式加進來處理後 還要補上 df_result？？
                                        # list_domain_name_remain是要傳給 五_1用的
                                        # 
                                        # hrefs_dict備用？範例:{21: 'http://moneydj.emega.com.tw/z/zc/zco/zco0/zco0.djhtm?a=1342&b=9600&BHID=9600',
                                        #                      25: 'http:// <...> 4&BHID=9300', 
                                        #                      91: 'http://moneydj.emega.com.tw/z/zc/zco/zco0/zco0.djhtm?a=1342&b=9134&BHID=9100'}
###############################################################
# 五_1:第二輪逐次連上後續幾家網站:
# 是被 五、呼叫
def retry_WebSites(value_of_domain_name_remain, hrefs_dict, df_result, stock_name_id): # 此時的 hrefs_dict是放不成功的
                                                             # 範例:{21: 'http://moneydj.emega.com.tw/z/zc/zco/zco0/zco0.djhtm?a=1342&b=9600&BHID=9600',
                                                             # 25: 'http:// <...> 4&BHID=9300', 
                                                             # 91: 'http://moneydj.emega.com.tw/z/zc/zco/zco0/zco0.djhtm?a=1342&b=9134&BHID=9100'}
    keys_for_pop = []

    for key,value in hrefs_dict.items():
        old_value =value.split('/z/zc/zco/zco0/zco0')[0]
        print('old_value=', old_value)
        hrefs_dict[key] = value.replace(old_value, value_of_domain_name_remain)
        print('key---value=', key, hrefs_dict[key])

        url = hrefs_dict[key]
        driver.get(url)    

        # if 'justdata.moneydj' in url or 'jdata.yuanta' in url or 'moneydj.emega' in url or 'fubon-ebrokerdj' in url :
        if 'justdata.moneydj' in url or 'jdata.yuanta' in url or 'moneydj.emega' in url :

            driver.find_element_by_xpath('//*[@id="SysJustIFRAMEDIV"]/table/tbody/tr[2]/td[2]/center/form/table/tbody/tr/td/table/tbody/tr[5]/td/select[1]/option[4]').click()
        elif 'newjust.masterlink' in url:
            driver.find_element_by_xpath('//*[@id="SysJustIFRAMEDIV"]/table/tbody/tr[2]/td[2]/center/table/tbody/tr/td/form/table/tbody/tr/td/table/tbody/tr[5]/td/select[1]/option[4]').click()
        else : # 'djinfo.cathaysec' in url:
            driver.find_element_by_xpath('//*[@id="SysJustIFRAMEDIV"]/table/tbody/tr[2]/td[2]/span/form/table/tbody/tr/td/table/tbody/tr[5]/td/select[1]/option[4]').click()

        html_source = driver.page_source # 在 近六十日資料 的頁面，抓取 page_source

        try :
            dfs = pd.read_html(html_source, header=[0])    

        # 以下copy自 五、====================================================
        # 建立許多df_for_merge並儲存為csv

                    # 這裡似乎還要用程式處理？？
        ##### 以下這段針對不同網站，讀取table時特殊的處理

            if 'newjust.masterlink' in value :
                df_for_merge = dfs_for_merge[4] # fubon-ebrokerdj好像不是3這張table
            elif 'justdata.moneydj' in value :
                if '日期' in dfs_for_merge[-1].columns:
                    df_for_merge = dfs_for_merge[-1] # 也有可能是[3]這張table嗎？做個try？
                elif '日期' in dfs_for_merge[3].columns:
                    df_for_merge = dfs_for_merge[3]
                else :
                    print('justdata.moneydj第{}筆查無資料了，用continue繼續下一個key'.format(key))
                    continue
            else :
                df_for_merge = dfs_for_merge[3] # fubon-ebrokerdj好像不是3這張table
            
            print('五_1的第{}筆 有抓取到df，正確嗎？先 pop()之…'.format(key))

            ### 呼叫程式執行 合併至df_result、存檔(這裡 五_1 的第二三四五六輪網站 某分點 已做完 )

            #=======================================================================================================
            # 呼叫 # 六_1、各分點的df_for_merge存成csv檔，合併至df_result
            print('六_1、呼叫 sourtout_And_SaveToCvs()')

            # 上面已取得 stock_name_id
            df_SourtOut_SaveToCsv  = sourtout_And_SaveToCvs(df_for_merge, key, stock_name_id, agent_name)

            #========================================================================================================
            # 呼叫 # 六_2、前三欄 與 各券商分點資料 合併
            # print('六_2、呼叫 merge_dfs() 前三欄 與 各券商分點資料 合併 : ')
            df_result = merge_dfs(df_result, df_SourtOut_SaveToCsv, agent_name)
                      
            keys_for_pop.append(key)  
        except :
            print('五_1的第{}筆抓取df時有誤…'.format(key))
            continue
    
    # for迴圈結東了，再一次做pop()
    for key in keys_for_pop:
        hrefs_dict.pop(key)
    print('一次整個做pop()後的hrefs_dict', hrefs_dict)
    
    return hrefs_dict, df_result

##### 以上這段針對不同網站，讀取table時特殊的處理

# 以上copy自 五、========================================================

###################################################
# 六_1、被五、呼叫 ，各分點買賣資料存成csv檔 從五、傳入 : df_for_merge, key, stock_name_id, agent_name
def sourtout_And_SaveToCvs(df_for_merge, key, stock_name_id, agent_name):  
    
    # (六要叫人做事應先準備好，傳給做事的六_1，且自己也要用啊！以下兩行複製到六中  )  取出 個股名稱_代號 及 分點名稱
    #title_agent_replaced = title_agent.replace('籌碼相關','').replace('券商分點個股進出','').replace(' ','').replace('－','_').replace('(','_').replace(')','')

    # 整理資料 設index 為 日期
    df_SourtOut_SaveToCsv = df_for_merge.reset_index(drop=True).set_index(['日期'])

    # 建立各分點資料夾 及 存成 csv檔
    path_csv_file = os.path.join(r'C:\Users\user\Desktop\練功房\stock_crawer_brokertrace_1\{}'.format(stock_name_id))
    os.makedirs(path_csv_file, exist_ok=True) # 建立個股資料夾

#### 取得 stock_name_id_no 僅在六_1用，故不需回傳出去
    stock_name_id_no = stock_name_id + '_{}'.format(key)
    
    df_SourtOut_SaveToCsv.to_csv(path_csv_file+'\{}_{}.csv'.format(stock_name_id_no, agent_name), encoding='utf-8-sig')
    # 應該是 中磊_5388_5_元大.csv 
    
    print('已存{}分點券商的 csv檔…'.format(agent_name))
    # print('看一下df_SourtOut_SaveToCsv的長相', df_SourtOut_SaveToCsv)
    
    return df_SourtOut_SaveToCsv 

# 六_2、被五、呼叫，前三欄 與 各券商分點資料 合併   
def merge_dfs(df_result, df_SourtOut_SaveToCsv, agent_name):
    
    try :
        # 僅取出 買賣超 欄位資料建立成df_Buy，並改column為 元大，再去合併
        df_Buy = pd.DataFrame(df_SourtOut_SaveToCsv['買賣超(張)']).rename(columns={'買賣超(張)':'{}'.format(agent_name)})
        df_result = pd.merge(df_result, df_Buy,  how='outer', right_index=True, left_index=True)[::-1].fillna('-')
    except TimeoutException:
        print('')
        
    # 整行 的反轉
    df_result = df_result.reindex(columns=df_result.columns[::-1])
    
    return df_result
    

#　八、建立個股資料夾 及 存成 csv檔 及存至sqlite3
def ResultDf_SaveToCsv(stock_name_id, df_result): # 
    
    # 存成 csv 檔
    path_csv_file = os.path.join(r'C:\Users\user\Desktop\練功房\stock_crawer_brokertrace_1\{}'.format(stock_name_id))
    os.makedirs(path_csv_file, exist_ok=True) # 建立個股資料夾
    df_result.to_csv(r'C:\Users\user\Desktop\練功房\stock_crawer_brokertrace_1\{}\{}.csv'.format(stock_name_id,stock_name_id), encoding='utf-8-sig')
    
    # 存至sqlite3
    ID_ForSql = 'ID_{}'.format(stock_id)
    path_sqlite = r'C:\Users\user\Desktop\練功房\stock_crawer_brokertrace_1\{}\{}.db'.format(stock_name_id, ID_ForSql)
    connection = sqlite3.connect(path_sqlite)
    df_result.to_sql('{}'.format(ID_ForSql), connection, if_exists='replace')

    return ID_ForSql
#############################################
# 九、讀取 sql 資料成 dataframe，加總每一colums，並依大小排序，等待存至 鴻海_2317_sorted.csv(本方法 可獨立出來用來排序)
def ReadSql_AddSum_Sort(ID_ForSql, stock_name_id):
    path_sqlite = r'C:\Users\user\Desktop\練功房\stock_crawer_brokertrace_1\{}\{}.db'.format(stock_name_id, ID_ForSql)
    connection = sqlite3.connect(path_sqlite)

    df_from_sql = pd.read_sql('SELECT * FROM {}'.format(ID_ForSql), connection, index_col='日期', parse_dates=['日期'])
    #df_from_sql = ReadSql_AddSum_Sort(ID_ForSql, stock_name_id)

    # ② column 改名(不改了)
    df_from_sql.reset_index(inplace=True)
    # df_from_sql.rename(columns={'漲跌%':'合計--->漲跌%'}, inplace=True)
    
    # ③ 設前三個欄index，資料就會從第四欄開始
    df_from_sql.set_index(['日期', '收盤', '漲跌%'], inplace=True)

    # ④ 計算 每一欄 的所有資料的 合計 放在list中
    list_total = []
    # 將 df_from_sql全部資料轉成數字
    df = df_from_sql.apply(pd.to_numeric,errors = 'coerce')
    #print('df...{}'.format(i), df)
    for i in range(0,len(df_from_sql.columns)): # 從資料的第1欄(即 0)開始
        # 作廢:df = df[df.iloc[:,i].notnull()]
        total = int(df.iloc[:,i][df.iloc[:, i].notnull()].sum())
        #print('total..{}'.format(i), total)
        list_total.append(total)
    print('list_total...{}'.format(i), list_total)
    
    
    # 好強的寫法，可加上另一個 level 的 column！
    # 還不了解是什麼意思！！已體會:前面的df.colunms是各欄位名稱形成的一個level的list，逗號後面加上相等長度的另一個level的list
    df_from_sql.columns = [list_total, df_from_sql.columns]
    
    # 排序 會根據第一層 column ，即 合計欄 排序
    df_from_sql = df_from_sql.sort_index(axis=1,ascending=False)
    
    # 變更column level，把 合計 改到第二層，網路找到終極解答！
    df_from_sql = df_from_sql.reorder_levels([1,0], axis=1)
  
    return df_from_sql # 已排序完成，待存至 鴻海_2317_sorted.csv
##########################################
# 十一、 將 已排序後 的成品存至 鴻海_2317_sorted.csv
# 存成 csv 檔
def SaveSortedCsv(stock_name_id):
    path_csv_file = os.path.join(r'C:\Users\user\Desktop\練功房\stock_crawer_brokertrace_1\{}'.format(stock_name_id))
    os.makedirs(path_csv_file, exist_ok=True) # 建立個股資料夾
    df_from_sql.to_csv(r'C:\Users\user\Desktop\練功房\stock_crawer_brokertrace_1\{}\{}_sorted.csv'.format(stock_name_id,stock_name_id), encoding='utf-8-sig')

# # 存至sqlite3 (排序後的不必存，因為之後隨時可能會增加分點券商進sql)
# ID_ForSql = 'ID_{}'.format(stock_id)
# path_sqlite = r'C:\Users\user\Desktop\練功房\stock_crawer_brokertrace\{}\{}.db'.format(stock_name_id, ID_ForSql)
# connection = sqlite3.connect(path_sqlite)
# df_from_sql.to_sql('{}'.format(ID_ForSql+'_finally'), connection, if_exists='replace')
#######################################################
# Main Part

# 在Main Part產生 driver
driver = webdriver.Chrome(executable_path='./chromedriver.exe') # 此時會出現空白的頁面

# 呼叫二_1、 裡面再呼叫二、
tuple_for_href, href_list, hrefs_dict = get_periods_AllAgent_href(tuple_for_href) # 取得所有的href，

# 四、 取得 日期 收盤價 漲跌幅 的 df   (在 # 六、的廻圈外就先叫了！再於六的迴圈內 做merge！)
df_result = get_date_close_UpDown(stock_id)



# 五、專攻:讀取 hrefs 做成 df_for_merge，並回傳未完成的 新的hrefs_dict
# 會呼叫 五_1、六_1、六_2
# 此cell從六、中獨立出來實驗:
#df_for_merge, hrefs_dict = build_df_for_merge(hrefs_dict)
list_domain_name_remain, hrefs_dict, df_result, stock_name_id = build_df_for_merge(hrefs_dict,list_domain_name_remain, df_result)

########以下先 mark ##########################################################################################################

# 呼叫六、取得各href的df、呼叫 # 六_1、各分點買賣資料存成csv檔、呼叫 # 六_2、前三欄 與 各券商分點資料 合併
#df_result, title_agent, agent_name, stock_name_id = ScratchDataFrame(driver, df_result, href_list)



# 呼叫八、建立個股資料夾 及 存成 csv檔 及存至sqlite3
ID_ForSql = ResultDf_SaveToCsv(stock_name_id, df_result)

# 呼叫 九、讀取 sql 資料成 dataframe，加總每一colums，並依大小排序，等待存至 鴻海_2317_sorted.csv(本方法 可獨立出來用來排序)
df_from_sql = ReadSql_AddSum_Sort(ID_ForSql, stock_name_id)

# 呼叫十一、 將 已排序後 的成品存至 鴻海_2317_sorted.csv
SaveSortedCsv(stock_name_id)



請輸入股票代號 : 
4919
https://djinfo.cathaysec.com.tw/z/zc/zco/zco_4919.djhtm
[(3, 'http://newjust.masterlink.com.tw/'), (2, 'http://moneydj.emega.com.tw/'), (1, 'http://jdata.yuanta.com.tw/'), (0, 'https://justdata.moneydj.com/')]
len(href_set總共有多少個 href... 87
hrefs_dict= {0: 'https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0031003100360058&BHID=1160', 1: 'https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0039004100390078&BHID=9A00', 2: 'https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=9800&BHID=9800', 3: 'https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0035003800350055&BHID=5850', 4: 'https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=2180&BHID=2180', 5: 'https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0037003700390065&BHID=7790', 6: 'https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=5850&BHID=5850', 7: 'https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0039

[*********************100%***********************]  1 of 1 completed


<ipython-input-1-a176ba1c22fc>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['漲跌%'] = a


五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0031003100360058&BHID=1160
某分點的key:0，df 已成功！！
title_agent是 日盛-宜蘭 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存日盛-宜蘭分點券商的 csv檔…
迴圈 0合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0039004100390078&BHID=9A00
某分點的key:1，df 已成功！！
title_agent是 永豐金-桃園 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存永豐金-桃園分點券商的 csv檔…
迴圈 1合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=9800&BHID=9800
某分點的key:2，df 已成功！！
title_agent是 元大證券 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存元大證券分點券商的 csv檔…
迴圈 2合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0035003800350055&BHID=5850
某分點的key:3，df 已成功！！
title_agent是 統一-南京 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存統一-南京分點券商的 csv檔…
迴圈 3合併完成！！！！！休

某分點的key:34，df 已成功！！
title_agent是 國票-台南 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存國票-台南分點券商的 csv檔…
迴圈 34合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=9634&BHID=9600
某分點的key:35，df 已成功！！
title_agent是 富邦-羅東 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存富邦-羅東分點券商的 csv檔…
迴圈 35合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0039003800310045&BHID=9800
某分點的key:36，df 已成功！！
title_agent是 元大-發財 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存元大-發財分點券商的 csv檔…
迴圈 36合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=1520&BHID=1520
某分點的key:37，df 已成功！！
title_agent是 瑞士信貸 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存瑞士信貸分點券商的 csv檔…
迴圈 37合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=1230&BHID=1230
某分點的key:38，df 

某分點的key:67，df 已成功！！
title_agent是 華南永昌-竹北 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存華南永昌-竹北分點券商的 csv檔…
迴圈 67合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=1380&BHID=1380
某分點的key:68，df 已成功！！
title_agent是 台灣匯立證券 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存台灣匯立證券分點券商的 csv檔…
迴圈 68合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0035003800350048&BHID=5850
某分點的key:69，df 已成功！！
title_agent是 統一-桃園 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存統一-桃園分點券商的 csv檔…
迴圈 69合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=0039003800390041&BHID=9800
某分點的key:70，df 已成功！！
title_agent是 元大-北投 對 新唐(4919)個股 單一券商歷史明細
六_1、呼叫 sourtout_And_SaveToCvs()
已存元大-北投分點券商的 csv檔…
迴圈 70合併完成！！！！！休息1秒
五、中，做driver.get(value)時的value是… https://djinfo.cathaysec.com.tw/z/zc/zco/zco0/zco0.djhtm?a=4919&b=1560&BHID